## Ensemble propagation in a MD simulation (thermodynamic reweighting)

In this extra notebook, we will demonstrate how to UQ works in MD simulations with `metatomic` models, and how thermodynamic reweighting can be performed.

In [ ]:
import ase.build
from metatomic.torch import ModelOutput
from metatomic.torch.ase_calculator import MetatomicCalculator
from ase.md.bussi import Bussi
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
import numpy as np

Let's initialize the bulk aluminum configuration, then set up an MD simulation with the stochastic velocity rescaling thermostat by Bussi et al.

We will run the MD simulation for 1000 steps. (You should closely observe what happens in the output!) Along the way, we will save all of our energy predictions, both the mean of the model, as well as the raw ensemble predictions.

In [ ]:
system = ase.build.bulk("Al", "fcc", a=4.05)
system = system * (3, 3, 3)  # 108 atoms
MaxwellBoltzmannDistribution(system, temperature_K=300)

calc = MetatomicCalculator("model-llpr.pt")
system.calc = calc

integrator = Bussi(system, timestep=4 * ase.units.fs, temperature_K=300, taut=100 * ase.units.fs)
integrator.run(300)  # equilibrate

nsteps = 1000
energies = []
ensemble_energies = []
integrator.attach(lambda: energies.append(system.get_potential_energy()))
integrator.attach(
    lambda: ensemble_energies.append(calc.run_model(system, outputs={"energy_ensemble": ModelOutput()})["energy_ensemble"].block().values.detach().cpu().numpy())
)

integrator.run(nsteps)

energies = np.array(energies)  # shape (nsteps,)
ensemble_energies = np.concatenate(ensemble_energies)  # shape (nsteps, n_ensemble_members)

Let's now compute the statistics _without_ reweighting:

In [ ]:
average_potential_energy_by_ensemble = ensemble_energies.mean(axis=0)

mean = np.mean(average_potential_energy_by_ensemble)
std = np.std(average_potential_energy_by_ensemble)
print(f"Thermodynamic average of potential energy: ({mean:.2f} ± {std:.2f}) eV")

However, this doesn't take into account that different members of the ensemble would have run different MD trajectories entirely! Hence, one must reweight the contribution from each configuration according to how much ``it should be contributing to the desired thermodynamic distribution'' for each ensemble member. This means that thermodynamic reweighting will also be necessary for uncertainty propagation. Below, we calculate the reweighted results, using two different approaches:

In [ ]:
# simple, naive reweighting
thermodynamic_weights = np.exp(-(ensemble_energies - energies[:, None]) / (ase.units.kB * 300))
thermodynamic_weights = thermodynamic_weights / np.sum(thermodynamic_weights, axis=0, keepdims=True)
average_potential_energy_by_ensemble = np.sum(ensemble_energies * thermodynamic_weights, axis=0)  # shape (n_ensemble_members,)

mean = np.mean(average_potential_energy_by_ensemble)
std = np.std(average_potential_energy_by_ensemble)
print(f"Thermodynamic average of potential energy (with reweighting): ({mean:.2f} ± {std:.2f}) eV")

In [ ]:
# computes reweighting with an approximate, but statistically efficient, cumulant expansion (recommended)
average_potential_energy_by_ensemble = ensemble_energies.mean(axis=0) + (
(ensemble_energies * (ensemble_energies-energies[:, None])).mean(axis=0) - 
ensemble_energies.mean(axis=0)*(ensemble_energies-energies[:, None]).mean(axis=0)
   ) / (ase.units.kB * 300)

mean = np.mean(average_potential_energy_by_ensemble)
std = np.std(average_potential_energy_by_ensemble)
print(f"Thermodynamic average of potential energy (with reweighting and cumulant expansion): ({mean:.2f} ± {std:.2f}) eV")